# < center> kappa系数简 介< /center>

### kappa系数是什么

- Kappa系数用于一致性检验，也可以用于衡量分类精度，kappa系数的计算是基于混淆矩阵的。  
> 分类问题中，这个一致性就是模型预测结果和实际分类结果的一致性
- kappa系数是一种衡量分类精度的指标。它是通过把所有地表真实分类中的像元总数（N）乘以混淆矩阵对角线（Xkk）的和，再减去某一类地表真实像元总数与该类中被分类像元总数之积对所有类别求和的结果，再除以总像元数的平方减去某一类地表真实像元总数与该类中被分类像元总数之积对所有类别求和的结果所得到的。
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;——from百度百科  
- kappa系数取值为-1到1之间  
- $kappa = \frac{p_o-p_e}{1-p_e}$
> 其中 $p_o$ 是整体精度，也就是acc  
> $p_e$ 是所有类别对应的实际与预测数量的乘积之总和除以样本总数的平方  

- kappa系数计算示例  

| 成绩评级 | 好 | 中 | 差 |
| :-: | :-: | :-: | :-: |
| 好 | a | b | c |
| 中 | d | e | f |
| 差 | g | h | i |  
$p_o = \frac{a+e+i}{\sum{a\cdots i}}$  
$p_e = \frac{(a+d+g)\times(a+b+c) + (b+e+h)\times(d+e+f) + (c+f+i)\times(g+h+i)}{(\sum{a\cdots i})^2}$  

### 为什么要使用kappa
在分类问题中，各个类别的样本数量往往不太平衡，这时模型就很容易偏向样本多的类别。此时可能出现整体acc挺高，但是部分类别完全不能被预测的现象。

- eg: 类别0有10个样本，类别1有90的样本，则将所有类别都预测为1，acc也有90%。但类别0就完全被“抛弃”了。 

这时需要一种能够抑制模型的“偏向性”的指标来代替acc。而kappa正好能够抑制模型的“偏向性”。

# <center> 编程计算kappa值 </center>

### 基于混淆矩阵，使用nunpy手动实现

In [59]:
import numpy as np

# 没有对输入的合法性进行校验
# 使用时需要注意
def kappa(confusion_matrix):
    """计算kappa值系数"""
    sum_row_list = np.sum(confusion_matrix, axis=0)
    sum_col_list = np.sum(confusion_matrix, axis=1)
    sum_total = sum(sum_col_list)
    pe = np.dot(sum_row_list, sum_col_list) / float(sum_total ** 2)
    po = np.trace(confusion_matrix) / float(sum_total)
    return (po - pe) / (1 - pe)

- #### 使用无偏向和有偏向的混淆矩阵分别测试

In [57]:
# 无偏向的混淆矩阵
balance_matrix = np.array(
    [
        [2,  1,  1],
        [1,  2,  1],
        [1,  1,  2]
    ]
)

# 有偏向的混淆矩阵
unbalance_matrix = np.array(
    [
        [0,  0,  3],
        [0,  0,  3],
        [0,  0,  3]
    ]
)

In [58]:
kappa_balance = kappa(balance_matrix)
print(kappa_balance)

kappa_unbalance = kappa(unbalance_matrix)
print(kappa_unbalance)

0.25
0.0


> **结论:**  上面两个混淆矩阵的acc一样，但是kappa值不同，偏向性强的kappa值低。

### sklearn中的kappa

In [63]:
from sklearn.metrics import cohen_kappa_score
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
kappa_value = cohen_kappa_score(y_true, y_pred)
print("kappa值为 %f" % kappa_value)

kappa值为 0.428571


### tensorflow中的kappa

### pytorch中的kappa